### Import all the libaries

In [102]:
#import all libaries
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Download the page

In [103]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page

<Response [200]>

### Parse Page with beautifulsoup

In [104]:
canada_postal_codes = []
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table')
table_body = table.find('tbody')

rows = table_body.findAll('tr')
for row in rows:
    cols = row.findAll('td')    
    cols = [ele.text.strip() for ele in cols]
    canada_postal_codes.append(cols)

# Remove the empty array from front
canada_postal_codes = canada_postal_codes[1:]
canada_postal_codes[0:5]

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront']]

### Convert the list to pandas Dataframe

In [105]:
columns = ["Postcode", "Borough", "Neighbourhood"]
canada_df = pd.DataFrame(canada_postal_codes, columns = columns)
canada_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Cleaning Data

In [106]:
# Drop rows with Not assigned
canada_df_cleaned = canada_df[canada_df.Borough != 'Not assigned']
canada_df_cleaned.reset_index(drop = True, inplace=True)
canada_df_cleaned.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Merge Neighbourhood with same post codes

In [107]:
canada_grouped = canada_df_cleaned.groupby('Postcode', as_index = False).agg(lambda x: ', '.join(set(x)))
canada_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Dealing with Not assigned neighborhood

In [108]:
canada_grouped[canada_grouped.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [113]:
for row in canada_grouped.itertuples():
    if row.Neighbourhood == 'Not assigned':
        canada_grouped.at[row.Index,'Neighbourhood'] = row.Borough        

In [114]:
canada_grouped.iloc[85]

Postcode                  M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 85, dtype: object

In [116]:
canada_grouped.shape

(103, 3)